In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

import anndata
import scipy
import joblib
import os

In [ ]:
file_name = '/workdir/data/adata.h5ad'
output_dir = '/workdir/results/'

adata = sc.read(file_name)

In [ ]:
adata.X = adata.layers['X']
#adata.X = adata.layers['logX']

hvgs_list = adata[:,adata.var.highly_variable==True].var.index.tolist()

QUERY = pd.DataFrame(data = adata[:,hvgs_list].X, 
                     index = adata[:,hvgs_list].obs['sample'].tolist(), 
                     columns = adata[:,hvgs_list].var.index.tolist())

In [ ]:
# QUERY is a n cells by hvg pandas dataframe with the cell sample as the index

print(QUERY.shape)
QUERY.head()

In [ ]:
# n and reps should be chosen based on the distribution of sample size

n = 2500
reps = np.arange(50)

This Next Code Block will take a significant amount of time, esepcially for a large number of samples and HVGs.
Recommended to run in the background over night when the notebook is not needed

In [ ]:
# GENE-GENE COVARIANCE WITHIN EACH PATIENT

logvolume_castle_dict = {}

for s in QUERY.index.unique().tolist():
    
    print(s)

    logvolume_castle_dict[s] = np.zeros(len(reps)) 
    
    for rep in reps: 
        
        print(rep)
        
        # Subset to current sample:
        SUBSET = QUERY.loc[s].sample(n=n,replace=True) 
        
        # Compute Gene Covariance:
        GENE_COV = SUBSET.cov(min_periods=None) 
        
        # Compute eigenvalues:
        eigenvalues = np.linalg.eigvals(GENE_COV) 
        
        # Compute logvolume:
        logvolume = float(np.sum(0.5*np.log10(eigenvalues[eigenvalues>0]**2))/len(eigenvalues)) 
        logvolume_castle_dict[s][rep] = logvolume 

In [ ]:
# WRITE RESULTS TO DATAFRAME 
boxplot_data_goi = pd.DataFrame() 
boxplot_data_goi['Sample'] = np.ravel([[x]*len(logvolume_castle_dict[x]) 
                                       for x in QUERY.index.unique().tolist()])

boxplot_data_goi['Log Phenotypic Volume'] = np.ravel([logvolume_castle_dict[x] 
                                                      for x in QUERY.index.unique().tolist()])

# STATISTIC 
Stat_dict = {}

for s in boxplot_data_goi['Sample'].unique().tolist():
    
    Stat_dict[s] = boxplot_data_goi.loc[boxplot_data_goi['Sample']==s]['Log Phenotypic Volume'].values 
    
    # Pickle results incase needed later: 
    joblib.dump(Stat_dict[s], output_dir+f'Phenotypic_Volume_{s}') 

The pickled phenotypic volume results above can be imported using joblib.load()

In [ ]:
# Visualize using violin plot:

fig, ax = plt.subplots(figsize=(7,7))

ax = sns.violinplot(x='Sample', y='Log Phenotypic Volume', data=boxplot_data_goi,
                    order=boxplot_data_goi.groupby('Sample').mean().\
                          sort_values(by='Log Phenotypic Volume').index.values,
                    #hue='Status',
                    #palette={'Alive': 'white', 'Deceased': 'gray'},
                    dodge=False,
                    scale='count')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xticks(rotation=90)

# SAVE FIGURE
figure_label = 'Phenotypic_Volume_Violin_by_Sample_count_scale'
fn = output_dir + figure_label
    
d = os.path.dirname(fn)
if not os.path.exists(d):
    os.makedirs(d)
    
fig.savefig(fn + '.png', bbox_inches='tight',dpi=400)
fig.savefig(fn + '.pdf', bbox_inches='tight',dpi=400)
print(fn)

Compute Mann Whitney U Test Between pairwise phenotyic volume distributions

In [ ]:
for i in range(len(boxplot_data_goi['Sample'].unique().tolist())):
    for j in range(i+1,len(boxplot_data_goi['Sample'].unique().tolist())):
        
        s1 = boxplot_data_goi['Sample'].unique().tolist()[i]
        s2 = boxplot_data_goi['Sample'].unique().tolist()[j]
        
        stat, p = scipy.stats.mannwhitneyu(Stat_dict[s1], Stat_dict[s2])
        
        print(f'{s1} vs {s2} :')
        print(f'Statistic = {stat}, p-value = {p}')
        print('')